<a href="https://colab.research.google.com/github/MRAMIRE/Tesis_Modelo_Deep_Learning_USA/blob/master/4_1_Entrenamiento_ADAM_Cross.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PROCESAMIENTO DE DATA:**

###FELIPE ANTONIO MELO CIFUENTES

###MARIA CAMILA RAMIREZ FERRO




##### Clonación de repositorio github

In [0]:
import os
if os.path.exists('Tesis_Modelo_Deep_Learning_USA/'):
  !git -C Tesis_Modelo_Deep_Learning_USA/ fetch --all
  !git -C Tesis_Modelo_Deep_Learning_USA/ reset --hard origin/master
  !git -C Tesis_Modelo_Deep_Learning_USA/ pull origin master;
else:
  !git clone https://github.com/MRAMIRE/Tesis_Modelo_Deep_Learning_USA.git
%cd Tesis_Modelo_Deep_Learning_USA/
from utils import *
from Unet import *

Cloning into 'Tesis_Modelo_Deep_Learning_USA'...
remote: Enumerating objects: 150, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 339 (delta 82), reused 90 (delta 38), pack-reused 189
Receiving objects: 100% (339/339), 2.24 MiB | 30.99 MiB/s, done.
Resolving deltas: 100% (168/168), done.
/content/Tesis_Modelo_Deep_Learning_USA


##### Permisos y cambio de directorio:

In [0]:
from google.colab import drive
#Permisos
drive.mount('/gdrive')
#Cambio de Directorio
%cd /gdrive/My\ Drive/Tesis
#%cd /gdrive/Share\Whit\Me
%ls 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive/My Drive/Tesis
'1. Exploración de los Datos'                           'Pruebas 9 12'/
'2. Visualización de la Data.ipynb'                      Resultados
'3. Pre-PorcesamientoV2-Data.Pre-PorcesamientoV2-Data'   Resultados_suma
'5. Resultados_cross'                                    runs/
'6. TensorBoard '                                        sample_submission.csv
 check/                                                  set_test.csv
'Data Set.ipynb'     

#####Importar librerias:

In [0]:
import numpy as np
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm_notebook as tqdm
import pylab as plt
from torch.autograd import Function
from torch import optim
from torch.utils.tensorboard import SummaryWriter
import logging
from torchvision import transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau



```
import matplotlib
from torchvision import utils
%matplotlib inline
import argparse
import sys
```



##### Eval

In [0]:
def eval_net(net, loader, device, n_val):
    """Evaluation without the densecrf with the dice coefficient"""
    net.eval()
    tot = 0
    with tqdm(total=n_val, desc='Validation round', unit='img', leave=False) as pbarV:
        for batch in loader:
            imgs = batch['image']
            true_masks = batch['mask']

            imgs = imgs.to(device=device, dtype=torch.float32)
            true_masks = true_masks.to(device=device, dtype=torch.float32)

            mask_pred = net(imgs)

            for true_mask, pred in zip(true_masks, mask_pred ):
                tot +=  dice_coeff(mask_pred, true_masks)
            pbarV.update(imgs.shape[0])
    return tot.item() / n_val


























































##### Def Train

In [0]:
def train_net(net,device,model_check,desc_check,epochs=5,batch_size=4,lr=0.1,save_cp=True,img_scale=0.5,n_workers=0):
  dir_checkpoint= model_check
  transform_img = transforms.Compose([transforms.Resize(350) , transforms.ToTensor()])
  train_dataset = CloudDataSet(csv_file ='set_train.csv', root_dir='./train_images/',transform = transform_img)
  n_train = len(train_dataset)
  val_dataset = CloudDataSet(csv_file ='set_test.csv', root_dir='./train_images/',transform = transform_img)
  n_val = len(val_dataset)
  n_data = n_val + n_train
  train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True, num_workers = n_workers)
  test_loader = DataLoader(val_dataset, batch_size =batch_size, shuffle=False, num_workers = n_workers)
  
  writer = SummaryWriter(comment=desc_check)
  global_step = 0
  logging.info(f'''Starting training:
        Epochs:          {epochs}
        Batch size:      {batch_size}
        Learning rate:   {lr}
        Training size:   {n_train}
        Validation size: {n_val}
        Checkpoints:     {save_cp}
        Device:          {device}
        Images scaling:  {img_scale}
    ''')
  net.to(device)
  optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay=1e-8)
  criterion = cross_entropy
  scheduler = ReduceLROnPlateau(optimizer, patience=3)
  no_val = 0
  for epoch in range(epochs):
    net.train()
    with tqdm(total=n_train, desc=f'Epoch {epoch + 1}/{epochs}', unit='img') as pbar:
      for batch in train_loader:
        imgs = batch['image']
        imgs = imgs.to(device=device,dtype=torch.float32)

        true_masks = batch['mask']
        true_masks = true_masks.to(device=device,dtype=torch.float32)
        masks_pred = net(imgs)

        loss = criterion(masks_pred, true_masks)
        dice_loss = dice_coeff(masks_pred, true_masks)
        #writer.add_scalar('Cross & Dice Loss/train', loss.item(), global_step)
        writer.add_scalar('Dice Loss/train',dice_loss.item(),global_step )

        pbar.set_postfix(**{'loss (batch)': loss.item()})

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        pbar.update(imgs.shape[0])
        global_step += 1
        
        if save_cp:
          try:
            os.mkdir(dir_checkpoint)
            logging.info('Created checkpoint directory')
          except OSError:
            pass
          torch.save(net.state_dict(),dir_checkpoint + f'CP_epoch{epoch + 1}.pth')
          logging.info(f'Checkpoint {epoch + 1} saved !')
    with torch.no_grad():
      val_score = eval_net(net, test_loader, device, n_val)
      logging.info('Validation cross entropy: {}'.format(val_score))
      writer.add_scalar('Dice Loss/test', val_score, global_step)
      no_val += 1
      print('Dice loss validacion #',no_val,': ',val_score)
      scheduler.step(val_score)
      writer.add_scalar('Learning Rate',optimizer.param_groups[0]['lr'],global_step )
      print(optimizer.param_groups[0]['lr'])

  writer.close()

##### Train

In [0]:
net = UNet(3,4,1)
model_check='check/ADAM-CROSS/'
desc_check= 'Op-ADAM_Cr-Cross_11-1'
train_net(net,'cuda',epochs=20,lr=0.1,model_check=model_check,desc_check=desc_check)

Dice loss validacion # 1 :  0.26479268641289655
0.1


Dice loss validacion # 2 :  0.2889306605420872
0.1


Dice loss validacion # 3 :  0.3060008999935224
0.1


Dice loss validacion # 4 :  0.31050388195915196
0.1


Dice loss validacion # 5 :  0.34274707840944935
0.010000000000000002


Dice loss validacion # 6 :  0.35470176207219944
0.010000000000000002


Dice loss validacion # 7 :  0.35880203082050965
0.010000000000000002


Dice loss validacion # 8 :  0.35557601257632254
0.010000000000000002


Dice loss validacion # 9 :  0.3589894928000518
0.0010000000000000002


Dice loss validacion # 10 :  0.3717635085278479
0.0010000000000000002
